# Step 1. 데이터 수집하기

패키지 임포트

In [ ]:
import tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

### 데이터 로딩: CSV 파일을 로드하고 필요한 데이터로 변환

데이터 로딩: ChatbotData.csv 파일을 로드하고 파싱하는 코드를 추가해야 합니다.
프로젝트의 기존 데이터 로딩 코드는 영화 대화 데이터셋을 로드하는데 사용됩니다. 이를 한국어 챗봇 데이터로 대체해야 합니다.
CSV 파일을 읽어들이고 적절한 형식으로 변환하는 작업이 필요합니다.

# Step 2. 데이터 전처리하기
전처리 함수: 문장에서 필요 없는 문자 제거 및 구두점 처리

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.strip()
    # 구두점과 한글 사이에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 한글과 구두점을 제외한 모든 문자를 공백으로 대체
    sentence = re.sub(r"[^가-힣?0-9?.!,]+", " ", sentence) #허용할 부분, 나머지 공백
    return sentence
print("슝=3")

In [ ]:
import pandas as pd

def load_conversations():
    data_path = 'data/ChatbotData.csv'
    data = pd.read_csv(data_path)
    inputs, outputs = [], []
    for _, row in data.iterrows():
      inputs.append(preprocess_sentence(row['Q']))
      outputs.append(preprocess_sentence(row['A']))
    return inputs, outputs

load_conversations()

In [ ]:
import pandas as pd

data_df = pd.read_csv("data/ChatbotData.csv")
data_df

# Step 4. 트랜스포머 모델 구성
# 포지셔널 인코딩 레이어: 각 단어의 위치에 따라 다른 신호를 모델에 제공

In [ ]:
# 포지셔널 인코딩 레이어 생성 => 단어를 인코딩을 하고 위치값을 더해주는것 
# 임베딩 행렬과 포지셔널 행렬이라는 두 행렬을 더함으로써 각 단어 벡터에 위치 정보를 더해주게 되는 것
# 같은 단어라고 하더라도 포지셔널 인코딩을 해준 경우에는 임베딩 벡터값이 달라지므로, 같은 단어라고 해도 
# 각각 다른 위치에 등장했다는 사실을 모델에 알려줄 수 있습니다.

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

print("슝=3")

In [ ]:
#행의 크기가 50, 열의 크기가 512인 행렬을 그려봅시다. 
#이를테면, 최대 문장의 길이가 50이고 워드 임베딩 차원을 512로 하는 모델의 입력 벡터 모양이 이와 같을 것입니다.
sample_pos_encoding = PositionalEncoding(50, 512) # Assuming PositionalEncoding is defined correctly

plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

# 스케일드 닷 프로덕트 어텐션 및 멀티 헤드 어텐션 구현

In [ ]:
# 스케일드 닷 프로덕트 어텐션 함수 => 단어 벡터간 유사도인데 소프트맥스를 적용
# Attention(Q,K,V)=softmax(QKT√dk)V𝐴𝑡𝑡𝑒𝑛𝑡𝑖𝑜𝑛(𝑄,𝐾,𝑉)=𝑠𝑜𝑓𝑡𝑚𝑎𝑥(𝑄𝐾𝑇𝑑𝑘)𝑉
# 내적(dot product)을 통해 "단어 벡터 간 유사도를 구한 후에" 
# 특정 값을 분모로 나눠주는 방식으로 Q와 K의 유사도를 구하였다고 하여
# 스케일드 닷 프로덕트 어텐션(Scaled Dot Product Attention) 이라고 합니다.

def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

print("슝=3")

In [ ]:
#어텐션을 병렬로 수행하는 것을 멀티 헤드 어텐션 => 분산처리
#내부적으로는 스케일드 닷 프로덕트 어텐션 함수를 호출하는 형태로 되어있음

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)  # [[YOUR CODE]]
    key = self.key_dense(key)  # [[YOUR CODE]]
    value = self.value_dense(value)  # [[YOUR CODE]]

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)  # [[YOUR CODE]]
    key = self.split_heads(key, batch_size)  # [[YOUR CODE]]
    value = self.split_heads(value, batch_size)  # [[YOUR CODE]]

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs
print("슝=3")

In [ ]:
#패딩 마스킹 작용, 패딩은 문장 길이를 맞춰주는 역활이고  패딩 마스킹은 이를 위해 숫자 0인 위치를 체크합니다.
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]
print("슝=3")
#데이터가 있는 곳은 0 빈곳은 1로 채워진 것을 확인
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

In [ ]:
# 룩 어헤드 마스킹(Look-ahead masking, 다음 단어 가리기) 
# 트랜스포머의 경우에는 문장 행렬을 만들어 한 번에 행렬 형태로 입력으로 들어간다는 특징이 있습니다. 그리고 이 특징 때문에 추가적인 마스킹(Masking) 을 필요
# 전체 문장이 문자 행렬로 들어가서 예측 가능하지만 시도하고자 하는 것은 이전 단어로부터 다음 단어 예측하는 훈련하는것
# 자신보다 다음 나올 단어를 참고하지 않도록 가리는 기법이 룩 어헤드 마스킹 기법

def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)
print("슝=3")

In [ ]:
# 테스트
print(create_look_ahead_mask(tf.constant([[1, 2, 3, 4, 5]])))
#숫자 0 넣어 테스트 
print(create_look_ahead_mask(tf.constant([[0, 5, 1, 5, 5]])))

번역기는 인코더와 디코더 두 가지 아키텍처로 구성돼 있었습니다. 인코더에 입력 문장이 들어가고, 디코더는 이에 상응하는 출력 문장을 생성합니다. 그리고 이를 훈련한다는 것은 결국 입력 문장과 출력 문장 두 가지 병렬 구조로 구성된 데이터셋을 훈련한다는 의미

### 훈련 데이터셋의 구성(번역)

입력 문장 : '저는 학생입니다.'
출력 문장 : 'I am a student'

### 훈련 데이터셋의 구성(질문-답변)

입력 문장 : '오늘의 날씨는 어때?'
출력 문장 : '오늘은 매우 화창한 날씨야

이런식으로 질문에 대한 대답이 구성된 데이터셋 or 입력된 문자에 사응하는 번역문자 등등
어떤 데이터 셋으로 인코더와 디코더를 구조하면 원하는 챗봇을 만드는 것

In [ ]:
# 인코더 층 만들기
# 셀프 어텐션은 멀티 헤드 어텐션으로 병렬적으로 이루어집니다.
# 두 개의 서브 층을 가지는 하나의 인코더 층을 구현하는 함수는 다음과 같습니다. 
# 함수 내부적으로 첫 번째 서브 층과 두 번째 서브 층을 구현

def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    # 패딩 마스크 사용
    # 패딩 마스크는 입력 데이터의 패딩 토큰을 무시하거나 
    # 마스킹하는 데 사용됩니다. 
    # 패딩 토큰은 의미 있는 정보를 전달하지 않으므로 
    # 자기주의 메커니즘의 계산에 영향을 주어서는 안 됩니다. 
    # 이 토큰들을 마스킹함으로써 모델은 실제 데이터에만 집중하여 
    # 주의 계산의 질과 관련성을 향상시킵니다.

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

# 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행

# 드롭아웃: 드롭아웃은 과적합을 방지하기 위해 사용되는 규제 기술입니다.
# 훈련 중에 드롭아웃은 무작위로 뉴런의 부분 집합을 무시하여
# 네트워크가 특정 특징의 부재에 강인하게 만들고 네트워크 뉴런 간의 독립성을 촉진합니다.
# 이는 보이지 않는 데이터에 대한 일반화를 개선할 수 있습니다.
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
# LayerNormalization: 이 기술은 배치 차원이 아닌 특징을 통해 입력을 정규화합니다. 
# 트랜스포머와 같은 깊은 네트워크에서 레이어 정규화는 학습 과정을 안정화하고
# 수렴에 필요한 훈련 단계 수를 줄여 훈련 속도를 높이는 데 도움을 줍니다.
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
# 완전 연결 계층: 주의 메커니즘 다음에 각 인코더에서 완전 연결 계층
# (또는 피드포워드 네트워크)은 주의 출력 특징 공간을 
# 더 높은 수준의 추상화를 포착하는 표현으로변환하는 데 사용됩니다.
# 이 계층은 모델에 비선형 능력을 도입할 수도 있어 더 복잡한 패턴을 학습할 수 있게 합니다.

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

드롭아웃과 레이어 정규화와 같은 기술은 과적합을 방지하고 모델의 훈련을 안정화하는 데 도움을 주는 동시에 전반적인 학습 역학을 개선합니다. 이는 언어 이해와 생성과 같은 작업에서 맥락과 미묘한 언어적 특징이 중요한 역할을 하는 데 있어 높은 성능을 달성하는 데 필수적입니다.

이러한 구성 요소를 인코더 레이어에 통합함으로써 트랜스포머 아키텍처는 시퀀스 간 작업을 효율적으로 처리하여 다양한 자연어 처리 응용 프로그램에서 뛰어난 성능을 달성합니다. 이 구성 요소들은 모델이 데이터의 효과적인 표현을 학습하도록 보장하며, 각 레이어는 입력 시퀀스의 더 깊은 이해와 처리에 기여합니다.

In [ ]:
#인코더 층을 쌓아 인코더 만들기

def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩 
  # 임베딩 행렬과 포지셔널 행렬이라는 두 행렬을 더함으로써 각 단어 벡터에 위치 정보를 더해주는것
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

In [ ]:
# 디코더 층 만들기
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

In [ ]:
#디코더 만들기
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

필요없을거같은 부분

In [ ]:
# 사용할 샘플의 최대 개수
#MAX_SAMPLES = 50000
#print(MAX_SAMPLES)

전처리 함수: 데이터셋에 맞게 전처리 함수를 조정해야 할 수 있습니다.
한글 텍스트 처리에 적합하도록 정규 표현식을 수정합니다.

In [ ]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations():
  id2line = {}
  with open(path_to_movie_lines, errors='ignore') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, 'r') as file:
    lines = file.readlines()

  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

    for i in range(len(conversation) - 1):
      # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
      inputs.append(preprocess_sentence(id2line[conversation[i]]))
      outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))

      if len(inputs) >= MAX_SAMPLES:
        return inputs, outputs
  return inputs, outputs
print("슝=3")

In [ ]:
def load_conversations():
    data_path = 'data/ChatbotData.csv'
    data = pd.read_csv(data_path)
    inputs, outputs = [], []
    for _, row in data.iterrows():
      inputs.append(preprocess_sentence(row['Q']))
      outputs.append(preprocess_sentence(row['A']))
    return inputs, outputs

load_conversations()

In [ ]:
# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장합니다.
questions, answers = load_conversations()
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

In [ ]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[20]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[20]))

In [ ]:
questions

# Step 3. 모델의 입력으로 사용할 수 있도록 데이터 인코딩

In [ ]:
import tensorflow_datasets as tfds
print("살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐")

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
print("슝=3 ")

In [ ]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

In [ ]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

In [ ]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

In [ ]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

In [ ]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40
print(MAX_LENGTH)

In [ ]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs
print("슝=3")

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

In [ ]:
# 교사 강요(Teacher Forcing) 사용하기
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
print("슝=3")

Step 4. 모델 구성하기
위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
print("슝=3")

In [ ]:
#모델생성
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

In [ ]:
#손실함수

def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)
print("슝=3")

In [ ]:
#커스텀 된 학습률
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
print("슝=3")

In [ ]:
sample_learning_rate = CustomSchedule(d_model=128)

plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

In [ ]:
#모델 컴파일
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
print("슝=3")

In [ ]:
# 훈련하기

EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

In [ ]:
# 챗봇 테스트하기

def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)
print("슝=3")

# Step 5. 모델 평가하기
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [ ]:
#의의 입력 문장에 대해서 decoder_inference() 함수를 호출하여 챗봇의 대답을 얻는 sentence_generation() 함수를 만듭니다.
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence
print("슝=3")

In [ ]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)
print("슝=3")

In [ ]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence
print("슝=3")

테스트

50포크

In [185]:
sentence_generation('Where have you been?')

입력 : Where have you been?
출력 : 사랑은 안 변하고 사람이 변해요 . 


'사랑은 안 변하고 사람이 변해요 . '

In [186]:
sentence_generation('너바보니?')

입력 : 너바보니?
출력 : 많이 만날 수 있을 거예요 . 


'많이 만날 수 있을 거예요 . '

In [187]:
sentence_generation('한글은 어려워')

입력 : 한글은 어려워
출력 : 같이 여행을 떠나보세요 . 


'같이 여행을 떠나보세요 . '

In [188]:
sentence_generation('너바보니?')

입력 : 너바보니?
출력 : 많이 만날 수 있을 거예요 . 


'많이 만날 수 있을 거예요 . '

In [189]:
sentence_generation('저는 위로해드리는 로봇이에요')

입력 : 저는 위로해드리는 로봇이에요
출력 : 많이 당황했겠어요 . 


'많이 당황했겠어요 . '

In [190]:
sentence_generation('1')

입력 : 1
출력 : 위로해 드립니다 . 


'위로해 드립니다 . '

In [191]:
sentence_generation('11')

입력 : 11
출력 : 위로해 드립니다 . 


'위로해 드립니다 . '

In [192]:
sentence_generation('2')

입력 : 2
출력 : 헤어진 하루를 세는 자신을 더 힘들게 만들 뿐이에요 . 


'헤어진 하루를 세는 자신을 더 힘들게 만들 뿐이에요 . '

In [193]:
sentence_generation('22')

입력 : 22
출력 : 헤어진 하루를 세는 자신을 더 힘들게 만들 뿐이에요 . 


'헤어진 하루를 세는 자신을 더 힘들게 만들 뿐이에요 . '

In [194]:
sentence_generation('100')

입력 : 100
출력 : 새로운 무언가를 해보는건 어떠세요 . 


'새로운 무언가를 해보는건 어떠세요 . '

In [195]:
sentence_generation('영화')

입력 : 영화
출력 : 영화는 현실과 다르지만 그런 사랑을 하도록 노력해보면 영화 같은 사랑 할 수 있을거예요 . 


'영화는 현실과 다르지만 그런 사랑을 하도록 노력해보면 영화 같은 사랑 할 수 있을거예요 . '

In [196]:
sentence_generation('음식점')

입력 : 음식점
출력 : 인생은 되돌이표인가봐요 . 


'인생은 되돌이표인가봐요 . '

In [197]:
sentence_generation('집에 가고 싶어')

입력 : 집에 가고 싶어
출력 : 집이 최고죠 . 


'집이 최고죠 . '

In [198]:
sentence_generation('부자가 되고 싶어')

입력 : 부자가 되고 싶어
출력 : 그런 걱정은 안 하셔도 되어요 . 


'그런 걱정은 안 하셔도 되어요 . '

### songys/Chatbot_data 을 활용하여 챗봇을 만들었어
결과를 보니 대화가 동문서답인 느낌이야, 이를 개선하기 위해선 어떠한 방법들이 존재할까?


### 현재 교과서처럼 들리는 챗봇의 대화 품질을 향상하려면 다음 전략을 고려하십시오.

더 많은 대화형 데이터 통합: 더욱 다양하고 대화형이며 비공식적인 데이터 소스로 교육 데이터 세트를 확장하여 챗봇에 보다 자연스러운 톤을 제공합니다. 현재 데이터 세트는 너무 형식적이거나 범위가 제한적일 수 있습니다.

상황별 이해 사용: 맥락을 이해하거나 대화 상태를 유지할 수 있는 알고리즘을 구현합니다. 이를 통해 챗봇은 보다 관련성이 높고 매력적인 응답을 생성할 수 있습니다.

NLU(자연어 이해) 통합: Google Dialogflow 또는 IBM Watson과 같은 도구는 사용자 의도를 보다 효과적으로 해석하는 이해 및 처리 계층을 추가하여 대화가 보다 자연스럽게 흐르도록 할 수 있습니다.

감정 분석 사용: 감정적인 내용에 대한 사용자 입력을 분석하고 공감과 적절한 감정적 반응을 반영하도록 반응을 맞춤화하여 상호 작용이 더욱 인간적인 느낌을 갖도록 만듭니다.

동적 응답 생성: 미리 정의된 정적인 응답 대신 신경망이나 생성 모델과 같은 기술을 사용하여 동적 응답을 생성합니다. GPT(Generative Pre-trained Transformer)와 같은 모델은 더욱 다양하고 인간과 유사한 텍스트를 생성할 수 있습니다.

정기 업데이트 및 피드백 루프: 새로운 데이터와 사용자 피드백으로 챗봇을 지속적으로 업데이트하여 응답을 개선하고 변화하는 사용자 기본 설정 및 언어 사용에 적응합니다.

대화 전략 개선: 유머 포함, 다양한 응답 길이, 복잡성, 반복적인 패턴 방지 등 흥미로운 대화를 만들기 위한 지침을 참조하세요[[4](https://www.phoneburner.com/ 블로그/재미있는 대화를 나누는 방법)].

이러한 변경 사항을 구현함으로써 챗봇은 대화에서 더욱 적응력이 뛰어나고 매력적이며 교과서와 같지 않게 될 수 있습니다.

배운 점 : 챗봇의 원리에 대해서 알 수 있었다.

아쉬운 점 : 결과물적으론 디테일한 답변이 안나오는게 아쉬웠다

느낀 점 : 어떻게하면 더 깊은 대화와 정확도를 올릴 수 있을지 고민되었다.

어려웠던 점 : 부족한 코딩 실력에 csv호출하고 연결하는 과정이 어려웠다
혼자서 이 문제만 2-3시간 넘게 해결해보았는데 팀원의 도움으로 간단하게 해결되었다
간단한 것에서 부딪치지 않도록 여러 상황을 겪어봐야겠다